# Irradiance analysis

This notebook allows for quick daily irradiance file analysis and graph output. It takes an excel file as input.

---

### Importing file

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
%matplotlib inline
from datetime import datetime

import requests
import csv
import time
import re
import datetime

from urllib.request import urlopen
from bs4 import BeautifulSoup

---

### Importing file


In [ ]:
columns = ['date','time','p1c','p2c']
crange1 = np.r_[0:2,15:17]
time_interval = 5

In the following cell the file path can be inputted:

In [ ]:
irradiance = pd.read_csv('./data/1205.tsv','\t') #INPUT TSV FILE NAME HERE


In [ ]:
irradiance= irradiance.iloc[:,0:4]
irradiance.columns = columns
irradiance = irradiance.loc[pd.notnull(irradiance['time'])]
irradiance['p1c']=irradiance['p1c'].apply(lambda x:x*0.948)
irradiance['p2c']=irradiance['p2c'].apply(lambda x:x*0.93)
irradiance['date']=pd.to_datetime(irradiance['date'])
irradiance['time'] = pd.to_datetime(irradiance['time'],format= '%H:%M:%S' ).dt.time
irradiance.head()

---

### Analysis

In [ ]:
ghi = round(irradiance['p1c'].sum()*time_interval/3600,3)
ghi

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
irradiance['p1cR'] = irradiance['p1c'].rolling(200, center=True).mean()
x_dt = [ datetime.datetime.combine(irradiance['date'][1], t) for t in irradiance['time'] ]
ax.plot(x_dt,irradiance['p1cR'], linewidth=2) # plotting
ax.fill_between(x_dt,irradiance['p1cR']) # filling under the curve


props = dict(boxstyle='round', facecolor='white', alpha=0.5)

# text box 
ax.text(0.73, 0.85, 'GHI: ' + str(ghi) + ' [Wh/m2]', transform=ax.transAxes, fontsize=12, bbox=props)
# title
ax.set_title("GHI for the day of "+ irradiance['date'][1].strftime("%m.%d.%Y"))

# axis labels
ax.set_ylabel('Irradiance [W/m2]', )
ax.set_xlabel('Time')
myFmt = mdates.DateFormatter('%H:%M')
ax.yaxis.grid( color='grey', linestyle='-', linewidth=0.3, alpha=0.9)
ax.xaxis.set_major_formatter(myFmt)

# output
fig.savefig("figures/"+ irradiance['date'][1].strftime("%m%d%y") + ".png")
plt.show()
